In [9]:
import sys
import os

# Go up one level (from notebooks/ to project root)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [22]:
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME", "o4-mini")

In [2]:
from PyPDF2 import PdfReader

def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

In [5]:
FILE_PATH = "../data/Handelsblatt_Artikel_Dobelli.pdf"

In [6]:
load_pdf(FILE_PATH)

'19.04.25, 17:36Geldanlage: „Je kompetenter Sie sind, desto konzentrierter sollte Ihr Depot sein“\nPage 1 of 15https://www.handelsblatt.com/finanzen/anlagestrategie/geldanlage-je…-sie-sind-desto-konzentrierter-sollte-ihr-depot-sein/100105520.htmlGeldanlage„Je kompetenter Sie sind,desto konzentrierter sollte IhrDepot sein“Rolf Dobelli hat viele Gesichter – das des Investorsist weniger bekannt. Hier verrät er erstmals, wie ersein Geld anlegt, wieso er weiter auf die USA setzt– und große Beträge nur bar zahlt.Benjamin Ansari18.04.2025 - 08:17 UhrArtikel anhören24:39\nRolf Dobelli: Mit Ehefrau und Zwillingssöhnen wohnt er in Bern. Foto: IMAGO/Newscom / El Pais\nFrankfurt. Steht ein schwerreicher Philosoph am Stehtisch und lästert über all dieFondsmanager, die sich um ihn scharen. So beginnt kein schlechter Witz, sondernder Nachmittag von Rolf Dobelli.An diesem Frühlingstag Anfang April steht der Schweizer Starautor in blauem Anzugmit Einstecktuch im Congress Center der Frankfurter Messe vo

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from src.config import OPENAI_API_KEY

embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

def embed_and_store(text, persist_directory=".chroma"):
    docs = splitter.create_documents([text])
    vectordb = Chroma.from_documents(docs, embedding, persist_directory=persist_directory)
    vectordb.persist()
    return vectordb


/var/folders/2x/vcryf4w947j2hgyynh9njl140000gn/T/ipykernel_1461/2874186048.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [34]:
persist_directory = ".chroma"

def setup_vectordb():
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    try:
        vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
        # Maybe also check if vectordb is empty here if you want
        print("VectorDB loaded from disk.")
    except Exception as e:
        print("No existing VectorDB found. You may need to ingest documents first.")
        raise e
    return vectordb

In [40]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from src.config import OPENAI_API_KEY

vectordb = setup_vectordb()

def ask(query, llm, vectordb):
    # embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    # vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
    retriever = vectordb.as_retriever()
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )
    result = qa.invoke({"query": query})

    answer = result.get("result", "No answer found.")
    source_docs = result.get("source_documents", [])
    print(source_docs)

    print("\nAnswer:\n" + answer)
    if source_docs:
        print("\nSources:")
        for i, doc in enumerate(source_docs):
            print(f"[{i+1}] {doc.metadata.get('source', 'Unknown source')}")
    else:
        print("\nNo sources found.")

    return answer

VectorDB loaded from disk.


In [ ]:
test_text = load_pdf(FILE_PATH)
embed_and_store(test_text)
vectordb = setup_vectordb()
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name=OPENAI_MODEL_NAME,
    temperature=1.0
)

ask("What is this interview about?", llm, vectordb)

/var/folders/2x/vcryf4w947j2hgyynh9njl140000gn/T/ipykernel_1461/2874186048.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


VectorDB loaded from disk.

Answer:
The piece is a profile-style interview with bestselling author and Unternehmer Rolf Dobelli. It covers:  
• His “Erfolgsrezept” – wie er arbeitet, sich auf ein Thema konzentriert und zur Zerstreuung Jazz hört  
• Seine Haltung zum ständigen „Self-deep-dive“ und warum er sich nicht bis ins letzte Detail analysieren muss  
• Das Zusammenspiel von Literatur und Wirtschaft in seinem Leben  
• Seine Vorträge zu News Avoidance, kognitiven Verzerrungen und Verhaltensökonomie  
• Und ganz praktisch: warum er gern in der Schweiz lebt und wie er seinen Alltag organisiert.

Sources:
[1] Unknown source
[2] Unknown source
[3] Unknown source
[4] Unknown source


'The piece is a profile-style interview with bestselling author and Unternehmer Rolf Dobelli. It covers:  \n• His “Erfolgsrezept” – wie er arbeitet, sich auf ein Thema konzentriert und zur Zerstreuung Jazz hört  \n• Seine Haltung zum ständigen „Self-deep-dive“ und warum er sich nicht bis ins letzte Detail analysieren muss  \n• Das Zusammenspiel von Literatur und Wirtschaft in seinem Leben  \n• Seine Vorträge zu News Avoidance, kognitiven Verzerrungen und Verhaltensökonomie  \n• Und ganz praktisch: warum er gern in der Schweiz lebt und wie er seinen Alltag organisiert.'

In [41]:
ask("In welche Firma empfiehlt Dobelli zu investieren?", llm, vectordb)

[Document(metadata={}, page_content='Page 14 of 15https://www.handelsblatt.com/finanzen/anlagestrategie/geldanlage-je…sie-sind-desto-konzentrierter-sollte-ihr-depot-sein/100105520.htmlNoch schlimmer. Nein, ich arbeite dann konzentriert ein Thema durch, das ichverstehen will. Und zur Zerstreuung höre ich Jazz.Wieso Dobelli so gern in der Schweiz lebtUm 16.50 Uhr ist Rolf Dobelli geschaﬀt. Er zieht sein blaues Sakko aus und lässt sichauf einem Sessel in der Kongresshalle nieder. Kurz hat er noch Zeit, dann muss erzurück nach Bern.'), Document(metadata={}, page_content='Branchentreﬀ für institutionelleAnleger aus der DACH-Region.Erst schimpft Dobelli aber über aktive Investoren. Er blickt in der Kongresshalleumher, zeigt auf die vielen Männer in Anzügen und wenigen Frauen in Blazern undsagt: „85 Prozent der Fondsmanager schlagen den Markt nicht – vernichten nur dasGeld ihrer Kunden.“Woran das liege? Weil bei aktiven Fonds Management-Fee und Performancegebührstark auf die Rendite drückten.

'Rolf Dobelli rät nicht in eine Einzel\xadaktie, sondern in einen ETF auf den US-Standard\xadindex S&P 500 zu investieren.'

In [ ]:
ask("In welche Firma hat Dobelli gerne investiert und betrachtet sie als einen Dauerbrenner?", llm, vectordb)


Answer:
Das weiß ich leider nicht.

Sources:
[1] Unknown source
[2] Unknown source
[3] Unknown source
[4] Unknown source


'Das weiß ich leider nicht.'